In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.linalg
import scipy
import copy

In [2]:
def train(params):
    alpha = params[0]
    nn_init_scale = params[1]

    train_num = 45
    seed = 58
    epochs = int(params[2])
    epoch_switch = 5000
    batch_size = train_num

    init_scale = 1
    noise_scale = 0.3
    #nn_init_scale = 10
    eta1 = 0
    eta2 = 1e-2
    weight_decay = 0
    log = 100
    w = 200
    wd1 = 0.0
    iters = []

    latents = []

    np.random.seed(seed)
    torch.manual_seed(seed)
    print("train_num={}".format(train_num))
    print("seed={}".format(seed))


    input_dim = 10
    latent_dim = 2
    output_dim = 30
    output_scale = 1
    p = 10
    x_templates = np.random.normal(0,1,size=(p, output_dim))
    y_templates = np.random.normal(0,1,size=(p, output_dim))*output_scale
    x_templates = torch.tensor(x_templates, dtype=torch.float, requires_grad=True)
    y_templates = torch.tensor(y_templates, dtype=torch.float, requires_grad=True)

    D0_id = []
    xx_id = []
    yy_id = []
    for i in range(p):
        for j in range(i,p):
            D0_id.append((i,j))
            xx_id.append(i)
            yy_id.append(j)

    xx_id = np.array(xx_id)
    yy_id = np.array(yy_id)

    base = 0
    for i in range(0,p):
        for j in range(i+1,p):
            for m in range(j,p):
                for n in range(m+1,p):
                    if (i+n-j-m) == 0:
                        base = base + 1


    # neural network part
    all_num = int(p*(p+1)/2)
    train_id = np.random.choice(all_num,train_num, replace=False)
    test_id = np.array(list(set(np.arange(all_num)) - set(train_id)))


    inputs_id = np.transpose(np.array([xx_id,yy_id]))
    out_id = (xx_id + yy_id) % p

    labels = torch.tensor(y_templates[out_id], dtype=torch.float, requires_grad=True)
    inputs = torch.cat([x_templates[xx_id],x_templates[yy_id]], dim=1)
    out_id = out_id

    labels_train = torch.tensor(y_templates[out_id[train_id]], dtype=torch.float, requires_grad=True)
    inputs_train = torch.cat([x_templates[xx_id[train_id]],x_templates[yy_id[train_id]]], dim=1)
    out_id_train = out_id[train_id]

    labels_test = torch.tensor(y_templates[out_id[test_id]], dtype=torch.float, requires_grad=True)
    inputs_test = torch.cat([x_templates[xx_id[test_id]],x_templates[yy_id[test_id]]], dim=1)
    out_id_test = out_id[test_id]



    # P0
    P0 = []
    P0_id = []

    ii = 0
    for i in range(all_num):
        for j in range(i+1,all_num):
            if np.sum(D0_id[i]) == np.sum(D0_id[j]):
                P0.append(frozenset({D0_id[i], D0_id[j]}))
                P0_id.append(ii)
                ii += 1

    P0_num = len(P0_id)

    # A
    A = []
    eq_id = 0

    for i1 in range(P0_num):
        i,j = list(P0[i1])[0]
        m,n = list(P0[i1])[1]
        if i+j==m+n:
            x = np.zeros(p,)
            x[i] = x[i] + 1; 
            x[j] = x[j] + 1; 
            x[m] = x[m] - 1;
            x[n] = x[n] - 1;
            A.append(x)
            eq_id = eq_id + 1

    A = np.array(A).astype(int)


    # P0(D)
    P0D_id = []

    ii = 0
    for i in range(all_num):
        for j in range(i+1,all_num):
            if np.sum(D0_id[i]) == np.sum(D0_id[j]):
                if i in train_id and j in train_id:
                    P0D_id.append(ii)
                ii += 1

    P0D = []
    for i in P0D_id:
        P0D.append(P0[i])

    # P0D_c
    P0D_c_id = set(P0_id) - set(P0D_id)


    # PD
    PD_id = []

    for i in P0D_c_id:
        P0D_id_aug = copy.deepcopy(P0D_id)
        P0D_id_aug.append(i)
        P0D_aug = []
        for j in P0D_id_aug:
            P0D_aug.append(P0[j])
        null_dim_1 = np.sum(np.linalg.eigh(np.matmul(np.transpose(A[P0D_id]),A[P0D_id]))[0] < 1e-8)
        null_dim_2 = np.sum(np.linalg.eigh(np.matmul(np.transpose(A[P0D_id_aug]),A[P0D_id_aug]))[0] < 1e-8)
        if null_dim_1 == null_dim_2:
            PD_id.append(i)

    PD_id = PD_id + P0D_id

    PD = []
    for i in PD_id:
        PD.append(P0[i])



    # Dbar(D)
    Dbar_id = list(copy.deepcopy(train_id))

    for i1 in test_id:
        flag = 0
        for j1 in train_id:
            i, j = D0_id[i1]
            m, n = D0_id[j1]
            if {(i,j),(m,n)} in PD:
                flag = 1
                break
        if flag == 1:
            Dbar_id.append(i1)

    acc_ideal = len(Dbar_id)/all_num
    acc_ideal_test = (len(Dbar_id)-len(train_id))/len(test_id)
    print("acc_ideal_test = {}/{}={}".format(len(Dbar_id)-len(train_id),len(test_id),acc_ideal_test))


    class NET(nn.Module):
        def __init__(self, input_dim, output_dim, w=200):
            super(NET, self).__init__()
            self.l1 = nn.Linear(input_dim, w)
            self.l2 = nn.Linear(w, w)
            self.l3 = nn.Linear(w, output_dim)

        def forward(self, x):
            #f = torch.nn.Tanh()
            f = torch.nn.LeakyReLU(0.2)
            self.x1 = f(self.l1(x))
            self.x2 = f(self.l2(self.x1))
            self.x3 = self.l3(self.x2)
            return self.x3

    class DEC(nn.Module):
        def __init__(self, input_dim, output_dim, w=200):
            super(DEC, self).__init__()
            self.net = NET(input_dim, output_dim, w=200)

        def forward(self, latent, x_id):
            self.add1 = latent[x_id[:,0]]
            self.add2 = latent[x_id[:,1]]
            self.add = self.add1 + self.add2
            self.out = self.net(self.add)
            return self.out


    class AE(nn.Module):

        def __init__(self, w=200, input_dim=1, output_dim=1):
            super(AE, self).__init__()
            self.enc = NET(input_dim, latent_dim, w=w)
            self.dec = DEC(latent_dim, output_dim, w=w)

        def forward(self, x, x_id):
            self.latent = self.enc(x)
            self.out = self.dec(self.latent,x_id)

            return self.out


    def L2(model):
        params = list(model.parameters())
        l2 = 0
        for i in range(6):
            if i == 0:
                params_flatten = params[i].reshape(-1,)
            params_flatten = torch.cat([params_flatten, params[i].reshape(-1,)])
        l2 = torch.sum(params_flatten**2)
        return params_flatten, l2

    def init(model, alpha):
        state_dict = model.state_dict()
        modules = ["l1.weight", "l1.bias", "l2.weight", "l2.bias", "l3.weight", "l3.bias"]
        for module in modules:
            state_dict[module] = state_dict[module] * alpha
        model.load_state_dict(state_dict)

    def init2(model, alpha):
        model.l1.weight.data = model.l1.weight * alpha
        model.l1.bias.data = model.l1.bias * alpha
        model.l2.weight.data = model.l2.weight * alpha
        model.l2.bias.data = model.l2.bias * alpha
        model.l3.weight.data = model.l3.weight * alpha
        model.l3.bias.data = model.l3.bias * alpha


    def grad(model):
        grads = list(student.parameters())
        for i in range(6):
            if i == 0:
                grad = grads[0].reshape(-1,)
            else:
                grad = torch.cat([grad, grads[i].reshape(-1,)])
        return grad


    def haha(a, b, c, d, e, f):
        p = [a.view(w,d_in), b, c.view(w,w), d, e.view(d_out, w), f]

        f = torch.nn.Tanh()
        x1 = f(F.linear(inputs_train, p[0], p[1]))
        x2 = f(F.linear(x1, p[2], p[3]))
        out = F.linear(x2, p[4], p[5])

        loss = torch.mean((out-labels_train), dim=1)
        return loss





    model = AE(input_dim=input_dim, output_dim=output_dim, w=w)
    init(model.dec.net, alpha=nn_init_scale)



    train_accs = []
    test_accs = []
    train_losses = []
    test_losses = []
    print("eta2={}".format(eta2))

    line = (2*torch.rand(p,latent_dim)-1)*init_scale*noise_scale
    line[:,0] = (2*torch.linspace(0,1,steps=p)-1)*init_scale
    random = (2*torch.rand(p,latent_dim)-1)*init_scale
    #alpha = 0.0
    latent = torch.nn.parameter.Parameter(alpha*random + (1-alpha)*line)

    optimizer1 = torch.optim.AdamW({latent}, lr=eta1, weight_decay = wd1)
    optimizer2 = torch.optim.Adam(model.dec.parameters(), lr=eta2, betas=(0.9,0.99))
    #optimizer2 = torch.optim.SGD(model.dec.parameters(), lr=eta2, momentum=0.)


    _, scale = L2(model.dec.net)
    
    loss_train_best = 1e4
    loss_test_best = 1e4


    for epoch in range(epochs): 

        if (epoch + 1) % epoch_switch == 0:
            optimizer2.param_groups[0]["lr"] = optimizer2.param_groups[0]["lr"]*0.2

        optimizer1.zero_grad()
        optimizer2.zero_grad()

        choice = np.random.choice(np.arange(train_num), batch_size, replace=False)
        
        # back propagation
        outputs_train = model.dec(latent, inputs_id[train_id])
        loss_train = torch.mean((outputs_train-labels_train)**2)
        loss_train.backward()
        optimizer1.step()
        optimizer2.step()


        # accuracy
        outputs_test = model.dec(latent, inputs_id[test_id])
        loss_test = torch.mean((outputs_test-labels_test)**2)

        pred_train_id = torch.argmin(torch.sum((outputs_train.unsqueeze(dim=1) - y_templates.unsqueeze(dim=0))**2, dim=2), dim=1)
        pred_test_id = torch.argmin(torch.sum((outputs_test.unsqueeze(dim=1) - y_templates.unsqueeze(dim=0))**2, dim=2), dim=1)

        acc_nn_train = np.mean(pred_train_id.detach().numpy() == out_id_train)
        acc_nn_test = np.mean(pred_test_id.detach().numpy() == out_id_test)

        train_accs.append(acc_nn_train)
        test_accs.append(acc_nn_test)
        train_losses.append(loss_train.detach().numpy())
        test_losses.append(loss_test.detach().numpy())
        
        # normalization: rescale weights such that the weight norm remains a constant in training.
        params, l2 = L2(model.dec.net)
        init2(model.dec.net, alpha=torch.sqrt(scale/l2))
        #params, l2 = L2(model.dec.net)

        if epoch % log == 0:
            print("epoch: %d  | train_loss: %.8f "%(epoch, loss_train.detach().numpy()))
            
        if loss_train.detach().numpy() < loss_train_best:
            loss_train_best = loss_train.detach().numpy()
            
        if loss_test.detach().numpy() < loss_test_best:
            loss_test_best = loss_test.detach().numpy()


    train_accs = np.array(train_accs)
    test_accs = np.array(test_accs)
    
    np.savetxt("./arithmetic_landscape_fine_55/train_alpha_%.3f_scale_%.3f"%(alpha, nn_init_scale), np.array([loss_train.detach().numpy()]))
    np.savetxt("./arithmetic_landscape_fine_55/test_alpha_%.3f_scale_%.3f"%(alpha, nn_init_scale), np.array([loss_test.detach().numpy()]))
    np.savetxt("./arithmetic_landscape_fine_55/trainbest_alpha_%.3f_scale_%.3f"%(alpha, nn_init_scale), np.array([loss_train.detach().numpy()]))
    np.savetxt("./arithmetic_landscape_fine_55/testbest_alpha_%.3f_scale_%.3f"%(alpha, nn_init_scale), np.array([loss_test.detach().numpy()]))


In [6]:
#alphas = np.linspace(0,0.2,num=21)
#scales = np.linspace(0,9,num=21)

alphas = np.linspace(0,0.1,num=21)
scales = np.linspace(2.5,4.0,num=21)

epochs = [15000]

xx, yy, zz = np.meshgrid(alphas, scales, epochs)
params = list(np.transpose(np.array([xx.reshape(-1,), yy.reshape(-1,), zz.reshape(-1,)])))

from multiprocess import Pool

if __name__ == '__main__':
    with Pool(10) as p:
        print(p.map(train, params))

train_num=54train_num=54

train_num=54seed=58seed=58


train_num=54seed=58

train_num=54seed=58

/var/folders/6j/b6y80djd4nb5hl73rv3sv8y80000gn/T/ipykernel_95751/53670628.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(y_templates[out_id], dtype=torch.float, requires_grad=True)
/var/folders/6j/b6y80djd4nb5hl73rv3sv8y80000gn/T/ipykernel_95751/53670628.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(y_templates[out_id], dtype=torch.float, requires_grad=True)



train_num=54


/var/folders/6j/b6y80djd4nb5hl73rv3sv8y80000gn/T/ipykernel_95751/53670628.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(y_templates[out_id], dtype=torch.float, requires_grad=True)
/var/folders/6j/b6y80djd4nb5hl73rv3sv8y80000gn/T/ipykernel_95751/53670628.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels_train = torch.tensor(y_templates[out_id[train_id]], dtype=torch.float, requires_grad=True)
/var/folders/6j/b6y80djd4nb5hl73rv3sv8y80000gn/T/ipykernel_95751/53670628.py:74: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(

seed=58
train_num=54

...

epoch: 9500  | train_loss: 0.00019162 
epoch: 9300  | train_loss: 0.00003460 
epoch: 9100  | train_loss: 0.00115438 
epoch: 12000  | train_loss: 0.00001159 
epoch: 8600  | train_loss: 0.00019656 
epoch: 10300  | train_loss: 0.00001095 
epoch: 10300  | train_loss: 0.00000227 
epoch: 11400  | train_loss: 0.00000454 
epoch: 9400  | train_loss: 0.00012930 
epoch: 9900  | train_loss: 0.00014979 
epoch: 10700  | train_loss: 0.00003129 
epoch: 9600  | train_loss: 0.00099411 
epoch: 9200  | train_loss: 0.00019189 
epoch: 12100  | train_loss: 0.00002834 
epoch: 8700  | train_loss: 0.00014155 
epoch: 10400  | train_loss: 0.00001635 
epoch: 10400  | train_loss: 0.00008284 
epoch: 9500  | train_loss: 0.00105205 
epoch: 9300  | train_loss: 0.00006003 epoch: 10000  | train_loss: 0.00082963 

epoch: 11500  | train_loss: 0.00001924 
epoch: 9700  | train_loss: 0.00008967 
epoch: 12200  | train_loss: 0.00000943 
epoch: 10800  | train_loss: 0.00000310 
epoch: 10500  | train_loss: 0.00003265 
epoch: 10500

epoch: 13100  | train_loss: 0.00004717 
epoch: 14400  | train_loss: 0.00000729 
epoch: 13700  | train_loss: 0.00000919 
epoch: 400  | train_loss: 0.18826489 
epoch: 13600  | train_loss: 0.00003629 
epoch: 13800  | train_loss: 0.00000412 
epoch: 12700  | train_loss: 0.00000478 
epoch: 14600  | train_loss: 0.00002271 
epoch: 14400  | train_loss: 0.00001075 
epoch: 13200  | train_loss: 0.00001062 
epoch: 1100  | train_loss: 0.02259673 
epoch: 14500  | train_loss: 0.00000571 
epoch: 500  | train_loss: 0.14240591 
epoch: 13800  | train_loss: 0.00003122 
epoch: 13700  | train_loss: 0.00002468 
epoch: 12800  | train_loss: 0.00001460 
epoch: 14500  | train_loss: 0.00001127 
epoch: 14700  | train_loss: 0.00001207 
epoch: 13900  | train_loss: 0.00000533 
epoch: 13300  | train_loss: 0.00000150 
epoch: 14600  | train_loss: 0.00002196 
epoch: 13900  | train_loss: 0.00002862 
epoch: 12900  | train_loss: 0.00002091 
epoch: 600  | train_loss: 0.09674645 
epoch: 1200  | train_loss: 0.07065307 
epoch: 1

epoch: 2800  | train_loss: 0.01693666 
epoch: 4900  | train_loss: 0.00758567 
epoch: 2900  | train_loss: 0.00846206 
epoch: 4600  | train_loss: 0.00193119 
epoch: 3600  | train_loss: 0.01602204 
epoch: 1800  | train_loss: 0.01282068 
epoch: 2100  | train_loss: 0.03285721 
epoch: 3600  | train_loss: 0.00557456 
epoch: 2700  | train_loss: 0.01695863 
epoch: 2900  | train_loss: 0.01554818 
epoch: 3500  | train_loss: 0.00982362 
epoch: 5000  | train_loss: 0.00172620 
epoch: 3000  | train_loss: 0.00976798 
epoch: 4700  | train_loss: 0.02712001 
epoch: 3700  | train_loss: 0.00836759 
epoch: 1900  | train_loss: 0.01907890 
epoch: 2200  | train_loss: 0.04901428 
epoch: 3700  | train_loss: 0.00256647 
epoch: 2800  | train_loss: 0.01751543 
epoch: 3000  | train_loss: 0.01960110 
epoch: 3100  | train_loss: 0.01351781 
epoch: 4800  | train_loss: 0.00482516 
epoch: 5100  | train_loss: 0.00072415 
epoch: 3600  | train_loss: 0.01806409 
epoch: 2000  | train_loss: 0.03467909 
epoch: 3800  | train_loss

epoch: 7200  | train_loss: 0.00011106 
epoch: 7800  | train_loss: 0.00005469 
epoch: 6800  | train_loss: 0.00075562 
epoch: 7000  | train_loss: 0.00133065 
epoch: 8800  | train_loss: 0.00121322 
epoch: 7900  | train_loss: 0.00003903 
epoch: 7700  | train_loss: 0.00020043 
epoch: 6300  | train_loss: 0.00032841 
epoch: 9000  | train_loss: 0.00041467 
epoch: 7300  | train_loss: 0.00023886 
epoch: 6000  | train_loss: 0.00054784 
epoch: 7900  | train_loss: 0.00257962 
epoch: 6900  | train_loss: 0.00071820 
epoch: 7100  | train_loss: 0.00024420 
epoch: 8900  | train_loss: 0.00060441 
epoch: 7800  | train_loss: 0.00009939 
epoch: 6400  | train_loss: 0.00058340 
epoch: 7400  | train_loss: 0.00014488 
epoch: 8000  | train_loss: 0.00013453 
epoch: 9100  | train_loss: 0.00011859 
epoch: 6100  | train_loss: 0.00131528 
epoch: 7000  | train_loss: 0.00016031 
epoch: 7200  | train_loss: 0.00262013 
epoch: 8000  | train_loss: 0.00017999 
epoch: 9000  | train_loss: 0.00008485 
epoch: 7500  | train_loss

epoch: 11800  | train_loss: 0.00000848 
epoch: 11600  | train_loss: 0.00001916 
epoch: 10100  | train_loss: 0.00000236 
epoch: 11300  | train_loss: 0.00000160 
epoch: 11900  | train_loss: 0.00000160 
epoch: 10400  | train_loss: 0.00001491 
epoch: 11900  | train_loss: 0.00000228 
epoch: 10900  | train_loss: 0.00008706 
epoch: 12900  | train_loss: 0.00001088 
epoch: 13200  | train_loss: 0.00003140 
epoch: 11900  | train_loss: 0.00006077 
epoch: 11700  | train_loss: 0.00000944 
epoch: 10200  | train_loss: 0.00000166 
epoch: 11400  | train_loss: 0.00001099 
epoch: 12000  | train_loss: 0.00000340 
epoch: 12000  | train_loss: 0.00001531 
epoch: 11000  | train_loss: 0.00002128 
epoch: 13300  | train_loss: 0.00000523 
epoch: 13000  | train_loss: 0.00002208 
epoch: 10500  | train_loss: 0.00002225 
epoch: 11800  | train_loss: 0.00001063 
epoch: 12000  | train_loss: 0.00002788 
epoch: 10300  | train_loss: 0.00000122 
epoch: 11500  | train_loss: 0.00000761 
epoch: 13400  | train_loss: 0.00000339 
